In [10]:
from importlib import reload
import json
import pretty_midi
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 422091078071127324
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13336856610618758248
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5588779008
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11742072890166847630
physical_device_desc: "device: 0, name: Tesla K20Xm, pci bus id: 0000:00:06.0, compute capability: 3.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 152799329061713978
physical_device_desc: "device: XLA_GPU device"
]


In [11]:
try:
    reload(midiMethods)
    from modules.midiMethods import *
except:
    print('reload failed')
    import modules.midiMethods as midiMethods
    from modules.midiMethods import *
try:
    reload(dataMethods)
    from modules.dataMethods import *
except:
    print('reload failed')
    import modules.dataMethods as dataMethods
    from modules.dataMethods import *
import modules.mlClasses as mlClasses 


In [12]:
import modules.models as models
reload(models)
import modules.mlClasses as mlClasses
reload(mlClasses)

<module 'modules.mlClasses' from 'C:\\Users\\Andrew\\Documents\\mlprojects\\380-music-representation\\modules\\mlClasses.py'>

In [175]:
# test out my function for going to note_bin representation
pm = pretty_midi.PrettyMIDI('midi/basic_test.mid')
noteB2 = pm2note_bin(pm)
print(noteB2)

[[27, 0, 0, 3, 19, 27], [0, 0, 0, 3, 20, 28], [5, 3, 35, 3, 14, 25], [32, 0, 0, 3, 16, 26], [2, 3, 27, 0, 21, 17], [34, 0, 0, 1, 13, 19], [3, 0, 42, 0, 20, 11], [5, 0, 38, 0, 21, 11], [32, 0, 3, 1, 20, 13], [2, 0, 42, 0, 28, 9], [7, 0, 59, 2, 16, 8], [31, 0, 2, 2, 15, 5]]


In [75]:
# if needed, read in filenames...
maestro = pd.read_csv('training_data/maestro-v2.0.0withPeriod.csv', index_col=0)
filenames = list(maestro['midi_filename'])


In [ ]:
# or process data set
data_path = 'C:/Users\Andrew/Documents/mlprojects/Datasets/MaestroV2.00/maestro-v2.0.0/'
examples = files2note_bin_examples(data_path, skip=1)
print(len(examples))

In [322]:
# store examples in json format
with open('training_data/training_data_note_bin_complete.json', 'w') as f:
    json.dump(examples, f)

In [13]:
# or read them in
with open('training_data/training_data_note_bin_complete.json', 'r') as f:
    examples = json.load(f)

In [15]:
# build simple model
# excellent example of recurrent model here https://www.tensorflow.org/tutorials/text/text_generation
hidden_state = 256
model2 = models.create_model1(hidden_state_size=hidden_state)
training_generator = mlClasses.DataGenerator(examples)

Model: "1layerLSTM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 6)]     0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 128, 88)      0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128, 10)      0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 128, 60)      0           input_1[0][0]                    
_________________________________________________________________________________________

In [16]:
model2.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# checkpoint = tf.keras.callbacks.ModelCheckpoint("weights/model1/{epoch:02d}-{train_loss:.2f}.hdf5", monitor='train_loss', verbose=2, save_best_only=True, save_weights_only=True)
model2.fit_generator(training_generator, epochs=10)
model2.save_weights('models/model2_10epochs256state.h5')

Epoch 1/10
 24/803 [..............................] - ETA: 13:54 - loss: 16.2674 - dense_loss: 4.2131 - dense_1_loss: 0.4972 - dense_2_loss: 3.4117 - dense_3_loss: 1.6432 - dense_4_loss: 3.3605 - dense_5_loss: 3.1417 - dense_accuracy: 0.0248 - dense_1_accuracy: 0.9378 - dense_2_accuracy: 0.1325 - dense_3_accuracy: 0.5572 - dense_4_accuracy: 0.0803 - dense_5_accuracy: 0.0653

KeyboardInterrupt: 

In [310]:
[Y[:,:,i] for i in range(6)][4]

array([[28, 13, 13, ..., 13, 15, 14],
       [14, 15, 22, ..., 11,  4, 25],
       [17, 22, 18, ..., 18, 17, 10],
       ...,
       [11, 11, 29, ..., 13,  3,  2],
       [ 0, 29, 28, ..., 17, 11,  9],
       [ 9,  8,  3, ..., 29, 29, 29]])

In [465]:
# This works just like this:
try:
    reload(modules.models)
except:
    print('reload failed')
    import modules.models as models

In [17]:
# tf.train.latest_checkpoint('weights\first_attempt')

prediction_model = models.create_model1(batch_size=1, stateful=True, hidden_state_size=hidden_state)

Model: "1layerLSTM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(1, 128, 6)]        0                                            
__________________________________________________________________________________________________
lambda_6 (Lambda)               (1, 128, 88)         0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_7 (Lambda)               (1, 128, 10)         0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_8 (Lambda)               (1, 128, 60)         0           input_2[0][0]                    
_________________________________________________________________________________________

In [18]:
prediction_model.load_weights('weights/model1_2epochs.h5')
new_seq = models.generate_music(prediction_model)
ns = [[int(a) for a in note] for note in new_seq]
print(ns)
pm = note_bin2pm(ns)
pm.write('midi/first_Tnotes1_0.2blahb.mid')

tf.Tensor([[[34  0  3  3  3 16]]], shape=(1, 1, 6), dtype=int32)
[[13, 5, 45, 7, 17, 3], [54, 0, 45, 6, 17, 5], [6, 8, 16, 4, 18, 13], [51, 6, 58, 7, 28, 27], [79, 4, 17, 17, 27, 22], [68, 8, 16, 9, 6, 20], [8, 7, 12, 16, 9, 13], [10, 0, 56, 5, 25, 28], [67, 3, 36, 6, 21, 13], [9, 1, 0, 9, 7, 2], [33, 6, 41, 16, 17, 31], [47, 2, 18, 6, 16, 16], [58, 9, 45, 7, 6, 13], [70, 5, 56, 10, 20, 11], [37, 1, 17, 7, 15, 4], [68, 1, 25, 14, 28, 30], [63, 0, 0, 2, 1, 7], [81, 9, 58, 8, 25, 16], [76, 8, 0, 1, 21, 14], [33, 6, 18, 9, 5, 28], [29, 8, 53, 9, 25, 4], [76, 8, 12, 16, 0, 12], [83, 7, 14, 6, 13, 23], [54, 4, 58, 14, 26, 22], [58, 8, 31, 8, 5, 8], [68, 1, 18, 4, 10, 11], [48, 9, 23, 17, 17, 2], [10, 7, 29, 9, 12, 29], [9, 9, 19, 9, 22, 31], [20, 0, 42, 17, 9, 4], [43, 6, 10, 13, 26, 30], [72, 6, 33, 5, 8, 3], [69, 0, 11, 5, 19, 4], [14, 6, 33, 2, 11, 28], [62, 2, 33, 4, 26, 25], [9, 0, 11, 14, 1, 4], [6, 6, 59, 4, 1, 2], [2, 5, 23, 9, 5, 13], [62, 1, 45, 2, 25, 4], [57, 2, 24, 13, 12, 10],